In [24]:
from jupyter_dash import JupyterDash
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
# import plotly.graph_objects as go
from dash import dcc

/usr/lib/python3/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [1]:
import sqlalchemy
from os import environ

import numpy as np
import pandas as pd

In [2]:
engine = sqlalchemy.create_engine("mariadb+mariadbconnector://"+environ.get("USER")+\
                                  ":"+environ.get("PSWD")+"@127.0.0.1:3306/nba")

In [3]:
A_cols = ['A.Name', 'PTS',
 'FGM', 'FGA', 'PM3', 'PA3', 'AST']

A_cols = ", ".join(A_cols)

In [4]:
fields = "* "

inner_join =  "Box_scores INNER JOIN Players on Box_scores.Player_ID = Players.ID) A "

inner_select = "(SELECT * FROM " + inner_join 

outer_join = "INNER JOIN Teams on Teams.ID=A.Team_ID "

outer_select = "SELECT " + A_cols +  ", Teams.Name as Team FROM " + inner_select  + outer_join

In [5]:
league = pd.read_sql(outer_select,engine)

In [7]:
def map_conference(df):
    west = ['NOP','UTA', 'MEM','PHX', 'POR', 'SAC', 'SAS', 'OKC','DAL', 'DEN', 'GSW', 'HOU','LAC', 'LAL','MIN']
    
    df['Conference'] = 'west'
    df['Conference'].where(df['Team'].isin(west),'east',inplace=True)
    return df

In [8]:
def map_division(df):
    divisions = {}
    # eastern conference divisions
    divisions['atlantic'] = ['BOS','BKN','NYK','PHI','TOR']
    divisions['central'] = ['CHI','CLE','DET','IND','MIL']
    divisions['southeast'] = ['ATL','CHA','MIA','ORL','WAS']
    
    # western conference divisions
    divisions['northwest'] = ['DEN', 'MIN','OKC','POR','UTA']
    divisions['pacific'] = ['GSW','LAC','LAL','PHX','SAC']
    divisions['southwest'] = ['DAL','HOU','MEM','NOP','SAS']
    
    df['Division'] = 'west'
    
    for i in divisions:
        df['Division'].where(~df['Team'].isin(divisions[i]),i,inplace=True)
    return df

In [9]:
league = map_division(league)
league = map_conference(league)

In [12]:
def calculate_features(df):
    df['eFG'] = 100*(df['FGM']+.5*df['PM3'])/df['FGA']
    df['PM2'] = df['FGM'] - df['PM3']

    return df

In [13]:
league = calculate_features(league)

In [15]:
league_avg = league.dropna().mean(numeric_only=True)

In [17]:
div_avg = league.groupby('Division').mean()

In [18]:
conf_avg = league.groupby('Conference').mean()

In [21]:
team_avg = league.groupby("Team").mean()

In [22]:
det = league.loc[league['Team']=='DET']

In [23]:
det_avg = det.mean(numeric_only=True)

In [25]:
def teams(names):
    options = []
    for i in names:
        d = {}
        d["label"] = i
        d["value"] = i
        options.append(d)
        
    return options

In [26]:
def opposition_select(opps, i):
    opts = team(opps)
    d = dcc.Dropdown(options=opts,value=None,id=i,multi=False)

In [38]:
opps = league.loc[league['Team']=='BKN'].groupby('Name').mean().sort_values('PTS',ascending=False)
opps['Team'] = 'BKN'
d = det.groupby('Name').mean().sort_values('PTS',ascending=False)
d['Team'] = 'DET'
df = pd.concat((d.head(10),opps.head(10)))

fig = px.scatter(y="PM3", x="PM2",color="eFG",size = "AST",data_frame = df,symbol='Team',text=df.index,
                 color_continuous_scale='RdBu_r')#RdYlBu_r
fig.update_layout(title ="Average 3PM vs 2PM", xaxis_title = "2PM",
                  title_x = 0.5,yaxis_title = "3PM",coloraxis_colorbar_x=1.15)

temp = det.groupby('Name').mean().sort_values('PM2',ascending=False)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.add_hline(league_avg['PM3'],annotation_text = 'League PM3 average',annotation_position='left top')
fig.add_vline(league_avg['PM2'],annotation={'text':'League PM2 average'})
other = det.groupby('Name').mean()


fig.update_traces(textposition='top center')



app = JupyterDash(__name__)


app.layout = html.Div(className="p-heading",
                      children=[html.H1(children="Detroit Pistons scoring"),
                                html.Div(children=[dcc.Graph(figure = fig, id = 'graph')])
])

# @app.callback(
#     Output('graph','figure'),
# #     Output('graph2','figure'),
# #     Output('graph3','figure'),
# #     Input('g1','value'))
# def update_figure(selected):
    
#     return fig

In [39]:
app.run_server(mode = "external")

/home/blunt/.local/lib/python3.7/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Dash app running on http://127.0.0.1:8050/


In [41]:
league.to_csv('Pistons_dash.csv')

In [ ]:
# fig.add_scatter(y=other["PM3"], x=other["PM2"],color=other["eFG"],size = other["AST"])#,
#                 color_continuous_scale='Blues')

# for k in df['Team'].unique():
#     temp = df.loc[df['Team']==k].sort_values("PTS",ascending=False)
#     for i in range(10):
#         fig.add_annotation(text=temp.index[i],x=temp.loc[temp.index[i],'PM2'],
#                            y=temp.loc[temp.index[i],'PM3'])


# fig = go.Figure()

# # Team averages
# temp = det.groupby('Name').mean()
# opps = league.loc[league['Team']=='BKN'].groupby('Name').mean()

# # size limit definition
# upper_limit = max(temp['AST'].max(),opps['AST'].max())
# sizeref = 2*upper_limit/100
# sizemin = 10

# colorbar=dict(tickvals = list(range(0,100,10)),ticktext=list(range(0,100,10)))

# # plot DET stats
# marker = dict(size=temp['AST'],sizeref=sizeref,sizemin=sizemin,
#               color=temp['eFG'],colorscale='Blues',showscale=True,colorbar=colorbar)

# fig.add_trace(go.Scatter(y=temp["PM3"], x=temp["PM2"],mode='markers',
#                          name='DET',marker=marker))

# # plot opposition stats
# marker = dict(size=opps['AST'],sizeref=sizeref,sizemin=sizemin,colorbar=colorbar,
#               color=opps['eFG'],colorscale='Reds',showscale=True,colorbar_x=1.15)

# fig.add_trace(go.Scatter(y=opps["PM3"], x=opps["PM2"],mode='markers',name='POR',marker = marker))

# # plot league averages
# fig.add_hline(y=league_avg['PM3'])
# fig.add_vline(x=league_avg['PM2'])